# Example

In [1]:
from physipy import units, s, m, asqarray
from physipy.qwidgets.qt import QuantityQtSlider
from physipy.quantity.utils import cached_property_depends_on

V = units["V"]
ohm = units["ohm"]
F = units["F"]

import numpy as np

import time

In [ ]:
def register_deps(name, deps):
    def decorator(f):
        f.deps = (name, deps)
        return f
    return decorator

class RegisteringType(type):
    def __init__(cls, name, bases, attrs):
        for key, val in attrs.items():
            deps = getattr(val, 'deps', None)
            if deps is not None:
                cls.curves[val.__name__] = deps

class ModelRC(metaclass=RegisteringType):
    
    curves = {}
    
    def __init__(self, Ve, R, C, u0=0*V, ech_t=np.arange(100)*s):
        self.R = R
        self.C = C
        self.Ve = Ve
        self.u0 = u0
        self.ech_t = ech_t
        
        # just add a params dict that describes the sliders...
        self.params = {
            "R"  :{"min":0*ohm, "max":10*ohm, "value":self.R},
            "C"  :{"min":0*F,   "max":10*F,   "value":self.C},
            "Ve" :{"min":0*V,   "max":10*V,   "value":self.Ve},
            "u0" :{"min":0*V,   "max":10*V,   "value":self.u0},
        }
        
    # ... and a method to provide all the curves you want to plot
    def get_curves(self):
        curves = {
            "xys_start_solved":{"xys":self.xy_response, "pen_color":"r",}, # "deps":["R", "C", "Ve", "u0"]
            "convergence_line":{"xys":self.xy_convergence, "pen_color":"g",},# "deps":["Ve"]},
            "slope_at_start":  {'xys':self.xy_slope_at_start, "pen_color":'b'},
        }
        return curves

    
    @cached_property_depends_on('R', 'C') # will not recompute if R and C state are unchanged
    def tau(self):
        #time.sleep(5)
        return self.R * self.C
    
    def xy_convergence(self):
        return asqarray([0*s, np.max(self.ech_t)]), asqarray([self.Ve, self.Ve])
    
    #@cached_property_depends_on('u0', 'Ve', "tau") # will not recompute if R and C state are unchanged
    def xy_slope_at_start(self):
        xs = asqarray([0*s, self.tau, self.tau])
        ys = asqarray([self.u0, self.Ve, self.u0])
        return xs, ys
    
    @register_deps("response", ["u0", "Ve", "R", "C"])
    def xy_response(self, ech_t=None):
        
        if ech_t is None:
            ech_t = self.ech_t

        xs = ech_t
        ys = (self.u0 - self.Ve) * np.exp(-ech_t/self.tau) + self.Ve
        return xs, ys 
    
    def xy_response_slow(self):
        #time.sleep(5)
        return self.xy_response()
        

In [ ]:
model = ModelRC(0*V, 0*ohm, 3*F)
model.curves

In [6]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QSlider, QWidget, QApplication, QVBoxLayout, QLabel, QMainWindow
from PyQt5.QtWidgets import QApplication, QHBoxLayout, QLabel, QSizePolicy, QSlider, QSpacerItem, \
    QVBoxLayout, QWidget, QLineEdit, QComboBox
import PyQt5.QtWidgets
import PyQt5.QtCore as QtCore



import pyqtgraph as pg
# Enable antialiasing for prettier plots
pg.setConfigOptions(antialias=True)
import pyqtgraph.console

class VuePyQt(QMainWindow):#QWidget):
    def __init__(self, model, parent = None):
        super(VuePyQt, self).__init__()

        self.model = model

        layout = QVBoxLayout()
        self.setContentsMargins(0, 0, 0, 0) 
        #self.setSpacing(0)
        
        # spent 3 days on this : https://stackoverflow.com/questions/2295290/what-do-lambda-function-closures-capture
        widgets = []
        for key, value in model.params.items():
            
            # create slider
            if hasattr(value["min"], "value"):
                slider = QuantityQtSlider(value["min"], 
                                          value["max"],
                                          value=value["value"], descr=key)
            else:
                slider = QuantityQtSlider(quantify(value["min"]), 
                                          quantify(value["max"]),
                                          value=quantify(value["value"]), descr=key)

            # add slider to Vue
            setattr(self, key+"_slider", slider)
            # connect slider's value to model's value
            getattr(self, key+"_slider").qtslider.valueChanged.connect(lambda qtvalue, key=key:self.set_attr(self.model, key))#(lambda qtvalue:self.update_model_param_value(qtvalue, slider, key))
            # make slider to update all curves
            #getattr(self, key+"_slider").qtslider.valueChanged.connect(lambda qtvalue:self.update_traces(qtvalue))
            
            # make slider to update dependent traces
            for k, v in self.model.curves.items():
                if key in v[1]: # loop over parameter list
                    func = getattr(self.model, k)
                    def _upd(qt_value):
                        xs, ys = func()
                        self.traces[name].setData(xs.value,ys.value)
                    getattr(self, key+"_slider").qtslider.valueChanged.connect(_upd)
            
            
            

            widgets.append(slider)
            
        for w in widgets:
            layout.addWidget(w)

        layout.setAlignment(QtCore.Qt.AlignTop)
        
        # to remove margins 
        layout.setContentsMargins(0, 0, 0, 0) #left top right bot
        # to remove space between each HBox
        layout.setSpacing(0)
        
        self.win = pg.GraphicsWindow(title="Basic plotting examples")

        self.canvas = self.win.addPlot(title="Plot11", row=1, col=1,)# axisItems={"bottom":sp_xaxis})
        self.canvas.setLabel('left', 'Y-axis Values')
        self.canvas.addLegend()
        self.canvas.showGrid(x=True, y=True)
        
        self.traces = dict()
        
        for name, trace_dict in self.model.get_curves().items():
            func = trace_dict["xys"]
            xs, ys = func()
            pen_color = trace_dict["pen_color"]
            self.trace(name, xs, ys, pen=pen_color)
        

        layout.addWidget(self.win)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

    def set_attr(self, obj, key):
        setattr(obj, key, getattr(self, key+"_slider").value)

    def update_model_param_value(self, qtvalue, slider, key):
        setattr(self.model, key, slider.value)
    
    def trace(self, name, dataset_x,dataset_y, pen="y", symbol="o"):
        if name in self.traces:
            self.traces[name].setData(dataset_x.value,dataset_y.value)
        else:
            self.traces[name] = self.canvas.plot(x=dataset_x.value, y=dataset_y.value, pen=pen, width=3, symbol=symbol, name=name, symbolBrush=pen)
            #pen =(0, 0, 200), symbolBrush =(0, 0, 200),
                      #symbolPen ='w', symbol ='o', symbolSize = 14, name ="symbol ='o'")
    
    def update_trace_generator(self, name, curve_func):
        def update_func():
            xs, ys = curve_func()
            self.trace(name, xs, ys)
        return update_func
    
    def update_traces(self, qtvalue):
        for name, trace_dict in self.model.get_curves().items():
            func = trace_dict["xys"]
            xs, ys = func()
            pen_color = trace_dict["pen_color"]
            self.trace(name, xs, ys, pen=pen_color)


In [ ]:
        
def main():
    app = QApplication(sys.argv)
    
    # to define a custom color
    from PyQt5.QtGui import QPalette
    palette = QPalette()
    palette.setColor(QPalette.ButtonText, Qt.red)
    app.setPalette(palette)
    
    Ve = 5 * V
    R  = 3 * ohm
    C  = 2 * F
    
    model = ModelRC(Ve, R, C)
    print(model.R, model.C, model.Ve, model.u0)

    vue = VuePyQt(model)
    vue.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

3.0 kg*m**2/(A**2*s**3) 2.0 A**2*s**4/(kg*m**2) 5.0 kg*m**2/(A*s**3) 0.0 V


/var/folders/5k/bf4syt7x1zjbhc6b28srzzym0000gn/T/ipykernel_78070/1489453334.py:73: DeprecationWarning: GraphicsWindow is deprecated, use GraphicsLayoutWidget instead,will be removed in 0.13
  self.win = pg.GraphicsWindow(title="Basic plotting examples")
